# Company's product area, revenue and customer base trends
### Summary of I/O csv files
Input Files
    > 'area.csv'     - product area code & descripton
    > 'caregory.csv' - product sub-category...
Output Files
    > 'company_area_percentage.csv'         - percentage revenue for all companies by ProductArea
    > 'company_category_percentage.csv'     - ... by MajorCategory
    > 'company_sales_revenue_36.csv'        - revenue for Company 36 in 36 months in 10 areas
    > 'company_sales_hh_num_36.csv'         - cumulative customer base for ...

## Use MongoDB to Store Source Data
Start MondoDB as a Windows Service from command line:

> $ "C:\Program Files\MongoDB\Server\3.4.2\bin\mongod.exe" --config "C:\Program Files\MongoDB\Server\3.4.2\mongod.cfg" --install

> $ net start MongoDB

Import csv files to db BT4221_DB 
> $ cd C:\Program Files\MongoDB\Server\3.4.2\bin

> $ mongoimport -d BT4221_DB -c Orders --type csv --file C:\Users\User\Desktop\BT4221\Project\11\DMEFOrders3Dataset2.csv --headerline

> $ mongoimport -d BT4221_DB -c Lines --type csv --file C:\Users\User\Desktop\BT4221\Project\11\DMEFLines3Dataset2.csv --headerline

> $ mongoimport -d BT4221_DB -c HouseholdZIP --type csv --file C:\Users\User\Desktop\BT4221\Project\11\4-DMEF3YrBase.csv --headerline


In [1]:
# Python version used
import sys
sys.version

'3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]'

In [2]:
""" Method for retriving data from mongodb"""
import pandas as pd
from pymongo import MongoClient

def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    return(conn[db])

def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    cursor = db[collection].find(query)
    df =  pd.DataFrame(list(cursor))
    # Delete the _id
    if no_id:
        del df['_id']
    return(df)

# 
## Describe all companies by product areas and subcategories

In [4]:
lines = read_mongo("BT4221_DB", "Lines")
print(len(lines))

(35536676, 9)


In [6]:
""" describe all companies by product area"""
lines.ProductArea = lines.ProductArea.apply(lambda s: s if s<10 else 10) # 10 product areas
df_area = lines.groupby(by=["CompanyID", "ProductArea"])['Dollars'].sum().to_frame().reset_index()
# pivot the dataframe
df_area_wide = df_area.pivot(index='CompanyID', columns='ProductArea', values='Dollars')

# change to % by revenue
import numpy as np
df_area_wide = df_area_wide.replace(np.nan, 0, regex=True)
s = df_area_wide.sum(axis=1)
df_area_perc = df_area_wide.loc[:,:].div(s, axis=0)

# change column names to actual areas
area = pd.read_csv("area.csv")
names = area.Description.head(10).tolist()
names = names[:-1]+["Others"]
df_area_perc.columns = names
print(df_area_perc.shape)
df_area_perc.head(2)

# Save results to csv file
file_name = "company_area_percentage.csv"
df_area_perc.to_csv(file_name)

print(df_area_perc.shape)
df_area_perc.sample(1)

print(df_area_perc.shape)
df_area_perc.sample(1)

(207, 10)


,Electronics,Entertainment,Fashion,Food & Entertaining,Health & Beauty,Home & Garden,Kids & Baby,Teens,Travel & Sports,Others
CompanyID,,,,,,,,,,
760,0.0,0.0,0.954521,0.0,0.0,0.0,0.0,0.0,0.00231,0.043169


In [7]:
""" company by product category"""
lines.MajorCategory = lines.MajorCategory.apply(lambda s: s if s<73 else 2) # change 1 and 73 to 2
lines.MajorCategory = lines.MajorCategory.apply(lambda s: s if s>1 else 2) # change 1 and 73 to 2

df_category = lines.groupby(by=["CompanyID", "MajorCategory"])['Dollars'].sum().to_frame().reset_index()
# pivot the dataframe
df_category_wide = df_category.pivot(index='CompanyID', columns='MajorCategory', values='Dollars')

# change to % by revenue
import numpy as np
df_category_wide = df_category_wide.replace(np.nan, 0, regex=True)
s = df_category_wide.sum(axis=1)
df_category_perc = df_category_wide.loc[:,:].div(s, axis=0)

# change column names to actual categorys
category = pd.read_csv("category.csv")
df_category_perc.rename(columns=dict(zip(category.Code, category.Description)), inplace=True)
print(df_category_perc.shape)
df_category_perc.sample(1)

# Save to csv files
file_name = "company_category_percentage.csv"
df_category_perc.to_csv(file_name)

(207, 69)


MajorCategory,Others,Gift Wrapping,Audio/music,Baby Furniture,Baby Gear,Bar & Cigar,Bath/closet,Bathroom Linens,Beauty,Bedroom Decor,...,Travel,Video/movies,Wall Decor,Window Treatments,Women's Apparel,Women's Fashion Accessories,Women's Footwear,Women's Jewelry,Women's Sports Apparel and Access,Women's Watches
CompanyID,,,,,,,,,,,,,,,,,,,,,
760,0.043169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.845774,0.0,0.027737,0.0,0.0,0.0


#
## Identify a company 
> with relative large number of unique customers in the training period

In [30]:
df1 = lines[lines.OrderDate<=20070630].groupby('CompanyID').agg({"HH_ID": 'nunique', 'Dollars': 'sum'})
df1 = df1.sort_values(by="HH_ID", ascending = False)
df1.head(3)

,Dollars,HH_ID
CompanyID,,
36,29074009,268068
153,29675748,242791
734,49838807,193985


### Thus, we select CompanyID = 36 as an example
# 
## Trends for Company 36
> Trend for Revenue in Each Product Area, by month

> Trend for cumulative customer base in each ProductArea, by month

In [33]:
company = 36
df = lines[lines.CompanyID==company]
df["month"] = df.OrderDate//100
print("number of lines:", len(df))
print("period: ",min(df.month),max(df.month))
df.sample(1)

number of lines: 2368670
period:  200501 200712


,HH_ID,CompanyID,OrderNum,OrderDate,ProductArea,MajorCategory,Dollars,Quantity,Channel,month
32313835,2087022,36,9055819,20060907,6,41,21,2,I,200609


In [36]:
"""group by month, 10 product areas, revenue as sum dollars"""
df2 = df.groupby(by=["ProductArea", "month"])['Dollars'].sum().to_frame().reset_index()
df2.sample(1)
# Save to csv
file_name = "company_sales_revenue_"+str(company)+".csv"
df2.to_csv(file_name, index=False)

,ProductArea,month,Dollars
175,5,200708,8865


In [39]:
"""group by month, 10 product areas, accumulative customer base = count hh_id"""
# For each month-ProductArea, count the number of unique customers in accumulative customer base
def num_hh_count(s):
    df_area = df.groupby('ProductArea').get_group(s.ProductArea)    
    df_month = df_area[df_area.month<=s.month]    
    hh_num = len(set(df_month.HH_ID)) # or df_month.HH_ID.nunique()
    return(hh_num)
df2["num_hh"] = df2.apply(num_hh_count, axis=1) 
df3 = df2[["ProductArea", "month", "num_hh"]]
df3.sample(1)
# Save to csv
file_name = "company_sales_hh_num_"+str(company)+".csv"
df3.to_csv(file_name, index=False)df3.sample(1)

,ProductArea,month,num_hh
74,3,200503,3111
